In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import datetime as dt
from technical.indicators import rsi
from technical.udemy_patterns import apply_patterns

/tmp/ipykernel_40836/3731595791.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
DF_M5 = pd.read_pickle(f"../data/candles/EUR_USD_M5.pkl")
DF_H1 = pd.read_pickle(f"../data/candles/EUR_USD_H1.pkl")

def break_down_candle(row, tp, sl, buy=True):
    start_date = row.time
    end_date = start_date + dt.timedelta(minutes=55)
    df = DF_M5[(DF_M5.time >= start_date)&(DF_M5.time <= end_date)]
    for index, row in df.iterrows():
        if buy:
            if row.bid_h >= tp:
                return 1,df,row.name
            elif row.bid_l <= sl:
                return 0, df,row.name
        else:
            if row.ask_l <= tp:
                return 1, df,row.name
            elif row.ask_h >= sl:
                return 0, df,row.name
            
candle_to_break = DF_H1.iloc[1]
DF_H1.iloc[1:2]

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
1,2018-01-02 01:00:00+00:00,1208,1.20168,1.20236,1.20143,1.20156,1.20161,1.20228,1.20135,1.20149,1.20175,1.20244,1.20151,1.20163


In [5]:
res, df, ind = break_down_candle(candle_to_break, 1.20726, 1.20140, buy=True)
print(res, ind)
df

0 23


,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
12,2018-01-02 01:00:00+00:00,112,1.20168,1.20192,1.20156,1.20190,1.20161,1.20184,1.20148,1.20184,1.20175,1.20199,1.20164,1.20197
13,2018-01-02 01:05:00+00:00,95,1.20192,1.20229,1.20184,1.20212,1.20184,1.20222,1.20176,1.20206,1.20199,1.20236,1.20191,1.20219
14,2018-01-02 01:10:00+00:00,154,1.20216,1.20232,1.20194,1.20224,1.20208,1.20225,1.20186,1.20216,1.20223,1.20239,1.20202,1.20233
15,2018-01-02 01:15:00+00:00,83,1.20222,1.20236,1.20216,1.20228,1.20214,1.20228,1.20208,1.20221,1.20231,1.20244,1.20224,1.20236
16,2018-01-02 01:20:00+00:00,98,1.20222,1.20231,1.20212,1.20223,1.20215,1.20223,1.20205,1.20216,1.20230,1.20239,1.20219,1.20230
17,2018-01-02 01:25:00+00:00,52,1.20223,1.20229,1.20210,1.20229,1.20215,1.20222,1.20202,1.20222,1.20231,1.20236,1.20217,1.20236
18,2018-01-02 01:30:00+00:00,96,1.20231,1.20234,1.20191,1.20192,1.20223,1.20226,1.20183,1.20184,1.20239,1.20241,1.20199,1.20199
19,2018-01-02 01:35:00+00:00,54,1.20192,1.20196,1.20184,1.20190,1.20185,1.20189,1.20176,1.20183,1.20199,1.20203,1.20191,1.20197
20,2018-01-02 01:40:00+00:00,91,1.20196,1.20214,1.20188,1.20197,1.20188,1.20205,1.20181,1.20188,1.20203,1.20222,1.20196,1.20206
21,2018-01-02 01:45:00+00:00,123,1.20196,1.20216,1.20189,1.20196,1.20188,1.20207,1.20181,1.20187,1.20204,1.20225,1.20197,1.20205


In [6]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE     
    
def apply_take_profit(row, PROFIT_FACTOR):
    if row.SIGNAL != NONE:
        if row.SIGNAL == BUY:
            if row.direction == BUY:
                return (row.ask_c - row.ask_o) * PROFIT_FACTOR + row.ask_c
            else:
                return (row.ask_o - row.ask_c) * PROFIT_FACTOR + row.ask_o
        else:
            if row.direction == SELL:
                return (row.bid_c - row.bid_o) * PROFIT_FACTOR + row.bid_c
            else:
                return (row.bid_o - row.bid_c) * PROFIT_FACTOR + row.bid_o
    else:
        return 0.0


def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        if row.SIGNAL == BUY:
            if row.direction == BUY:
                return row.ask_o
            else:
                return row.ask_c
        else:
            if row.direction == SELL:
                return row.bid_o
            else:
                return row.bid_c
    else:
        return 0.0

In [7]:
class Trade:
    DF_M5 = pd.read_pickle(f"../data/candles/EUR_USD_M5.pkl")
    def __init__(self, row, profit_factor, loss_factor):
        self.running = True
        self.start_index_m5 = row.name
        self.profit_factor = profit_factor
        self.loss_factor = loss_factor
        
        if row.SIGNAL == BUY:
            self.start_price = row.start_price_BUY
            self.trigger_price = row.start_price_BUY
            
        if row.SIGNAL == SELL:
            self.start_price = row.start_price_SELL
            self.trigger_price = row.start_price_SELL
            
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price

    def break_down_candle(self, row, buy=True):
        start_date = row.time
        end_date = start_date + dt.timedelta(minutes=55)
        df = DF_M5[(DF_M5.time >= start_date)&(DF_M5.time <= end_date)]
        for index, row in df.iterrows():
            if buy:
                if row.bid_h >= self.TP:
                    return 1, row.bid_h
                elif row.bid_l <= self.SL:
                    return 0, row.bid_l
            else:
                if row.ask_l <= self.TP:
                    return 1, row.ask_l
                elif row.ask_h >= self.SL:
                    return 0, row.ask_h
        
    def update(self, row):
        if self.SIGNAL == BUY:
            if row.bid_h >= self.TP and row.bid_l <= self.SL:
                res, trigger_price = self.break_down_candle(row)
                if res:
                    self.close_trade(row, self.profit_factor, trigger_price)
                else:
                    self.close_trade(row, self.loss_factor, trigger_price)
            elif row.bid_h >= self.TP:
                self.close_trade(row, self.profit_factor, row.bid_h)
            elif row.bid_l <= self.SL:
                self.close_trade(row, self.loss_factor, row.bid_l)
        if self.SIGNAL == SELL:
            if row.ask_l <= self.TP and row.ask_h >= self.SL:
                res, trigger_price = self.break_down_candle(row, buy=False)
                if res:
                    self.close_trade(row, self.profit_factor, trigger_price)
                else:
                    self.close_trade(row, self.loss_factor, trigger_price) 
            elif row.ask_l <= self.TP:
                self.close_trade(row, self.profit_factor, row.ask_l)
            elif row.ask_h >= self.SL:
                self.close_trade(row, self.loss_factor, row.ask_h)   

In [8]:
def apply_signals(df, PROFIT_FACTOR, sig):
    df["SIGNAL"] = df.apply(sig, axis=1)
    df["TP"] = df.apply(apply_take_profit, axis=1, PROFIT_FACTOR=PROFIT_FACTOR)
    df["SL"] = df.apply(apply_stop_loss, axis=1)


class GuruTester:
    def __init__(self, df_big,
                    apply_signal, 
                    df_m5,
                    LOSS_FACTOR = -1.0,
                    PROFIT_FACTOR = 1.5,
                    time_d=1 ):
        self.df_big = df_big.copy()
        self.apply_signal = apply_signal
        self.df_m5 = df_m5.copy()
        self.LOSS_FACTOR = LOSS_FACTOR
        self.PROFIT_FACTOR = PROFIT_FACTOR
        self.time_d = time_d

        self.prepare_data()
        
    def prepare_data(self):
        
        print("prepare_data...")

        apply_signals(self.df_big,
                    self.PROFIT_FACTOR,
                    self.apply_signal)

        self.df_big.rename(columns={
            'bid_c' : 'start_price_BUY', # for tracking the trade  
            'ask_c' : 'start_price_SELL',
            'm5_start' : 'time'
            }, inplace=True)
        print(self.df_big[self.df_big.SIGNAL != NONE].shape)


    def run_test(self):
        print("run_test...")
        open_trades_m5 = []
        closed_trades_m5 = []

        for index, row in self.df_big.iterrows():
            for ot in open_trades_m5:
                ot.update(row)
                if ot.running == False:
                    closed_trades_m5.append(ot)
            open_trades_m5 = [x for x in open_trades_m5 if x.running == True]
            if row.SIGNAL != NONE:
                open_trades_m5.append(Trade(row, self.PROFIT_FACTOR, self.LOSS_FACTOR))  

        self.df_results = pd.DataFrame.from_dict([vars(x) for x in closed_trades_m5]) 
        print("Result:", self.df_results.result.sum())

In [33]:
def run_pair(pair):
    df_an = pd.read_pickle(f"../data/candles/{pair}_H1.pkl").iloc[:1500]
    df_m5 = pd.read_pickle(f"../data/candles/{pair}_M5.pkl")
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)
    df_an = rsi(df_an)
    df_an = apply_patterns(df_an)
    df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()
    our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
            'bid_o', 'bid_h', 'bid_l', 'bid_c', 
            'ask_o', 'ask_h', 'ask_l', 'ask_c',
            'ENGULFING', 'direction', 'EMA_200', 'RSI_14' ]
    df_slim = df_an[our_cols].copy()
    df_slim.dropna(inplace=True)
    
    df_slim.reset_index(drop=True, inplace=True)
    gt = GuruTester(
        df_slim,
        apply_signal,
        df_m5,
    )
    
    gt.run_test()
    return gt.df_results

In [34]:
res = []
for p in ["EUR_USD"]:
    res.append(dict(pair=p,res=run_pair(p)))

prepare_data...
(3413, 20)
run_test...
Result: -1033.0


In [35]:
res[0]['res'].shape

(3413, 12)

In [27]:
for r in res:
    print(r['pair'], r['res'].result.sum())

EUR_USD -1016.0
GBP_JPY -1057.5


In [ ]:
EUR_USD -1016.0
GBP_JPY -1057.5